<a href="https://colab.research.google.com/github/PYH1107/generative_ai/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========== Cell 1: 解決依賴衝突並安裝必要套件 ==========
print("🔧 正在解決依賴衝突並安裝套件...")

# Step 1: 重新安裝相容的 PyTorch 生態系統
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124

# Step 2: 安裝其他 AI 相關套件（指定相容版本）
!pip install transformers==4.44.0 accelerate==0.34.0 safetensors==0.4.5

# Step 3: 安裝 diffusers（使用相容版本）
!pip install diffusers==0.30.0

# Step 4: 安裝 Gradio
!pip install gradio==4.44.0

# Step 5: 安裝 aisuite（只安裝需要的 provider）
!pip install groq==0.9.0 openai==1.35.8

# Step 6: 安裝 HuggingFace Hub
!pip install huggingface_hub==0.25.0

print("✅ 套件安裝完成！如果還有錯誤，請重啟 Runtime 後重新執行")

In [ ]:
# ========== Cell 2: 導入必要庫和簡化版 AI 客戶端 ==========
import torch
import gc
import random
import gradio as gr
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler
import requests
import json
import os
from google.colab import userdata
print("✅ 所有庫導入成功！")

# 簡化版 AI 客戶端（避免 aisuite 依賴問題）
class SimpleAIClient:
    def __init__(self, provider="groq", api_key=None):
        self.provider = provider
        self.api_key = api_key

        if provider == "groq":
            self.base_url = "https://api.groq.com/openai/v1/chat/completions"
            self.model = "llama3-70b-8192"
        elif provider == "openai":
            self.base_url = "https://api.openai.com/v1/chat/completions"
            self.model = "gpt-4o-mini"

    def get_response(self, messages):
        """獲取 AI 回應"""
        try:
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }

            data = {
                "model": self.model,
                "messages": messages,
                "temperature": 0.7,
                "max_tokens": 500
            }

            response = requests.post(self.base_url, headers=headers, json=data, timeout=30)

            if response.status_code == 200:
                result = response.json()
                return result["choices"][0]["message"]["content"]
            else:
                return f"API 錯誤: {response.status_code} - {response.text}"

        except requests.exceptions.Timeout:
            return "請求超時，請稍後再試"
        except requests.exceptions.RequestException as e:
            return f"連接錯誤: {str(e)}"
        except Exception as e:
            return f"未知錯誤: {str(e)}"

print("🤖 簡化版 AI 客戶端就緒！")


In [ ]:
# ========== Cell 3: API 金鑰設定 ==========
# 設定 AI Agent 的 API
try:
    api_key = userdata.get('groq')
    ai_client = SimpleAIClient(provider="groq", api_key=api_key)
    print("🔑 使用 Groq API")
except:
    print("❌ 未找到 Groq API Key")

In [ ]:
# ========== Cell 4: 檢測環境並加載 SD 模型 ==========
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ 檢測到設備: {device}")

# 加載 Stable Diffusion 模型
try:
    model_name = "runwayml/stable-diffusion-v1-5"
    pipe = StableDiffusionPipeline.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        use_safetensors=True
    ).to(device)

    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

    if device == "cpu":
        pipe.enable_attention_slicing()
        print("CPU 模式：已啟用最佳化")
    else:
        pipe.enable_model_cpu_offload()
        print("GPU 模式：已啟用記憶體最佳化")

    print("✅ Stable Diffusion 模型載入成功！")

except Exception as e:
    print(f"❌ 模型載入失敗: {e}")


In [ ]:
# ========== Cell 5: AI Agent 系統設計 ==========
class DiaryAgent:
    def __init__(self, ai_client):
        self.ai_client = ai_client
        self.conversation_history = []
        self.user_responses = []

        # 預設的問題序列
        self.questions = [
            "嗨！今天想和我聊聊嗎？先告訴我，今天你的心情如何呢？😊",
            "聽起來很棒！那今天有發生什麼特別的事情嗎？可以是開心的、有趣的，或是讓你印象深刻的事 ✨",
            "真有意思！如果要用一個顏色來形容現在的你，你會選什麼顏色？為什麼呢？🎨",
            "好棒的選擇！那你覺得現在的自己更像是什麼？比如一隻動物、一種植物，或是任何你想到的東西？🌟",
            "最後一個問題囉～如果我要畫一幅畫來展現今天的你，你希望自己在畫中是什麼樣子？在做什麼呢？🖼️"
        ]

        self.current_question_index = 0

    def get_ai_response(self, system_prompt, user_input):
        """獲取 AI 回應"""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]

        return self.ai_client.get_response(messages)

    def get_next_question(self):
        """獲取下一個問題"""
        if self.current_question_index < len(self.questions):
            question = self.questions[self.current_question_index]
            self.current_question_index += 1
            return question, False  # False 表示對話未結束
        else:
            return "謝謝你和我分享這麼多！現在讓我為你創作一幅專屬的畫像吧 🎨✨", True  # True 表示對話結束

    def process_user_response(self, user_input):
        """處理用戶回應並生成適當的跟進"""
        if self.current_question_index == 0:
            # 第一個問題的回應
            system_prompt = """你是一位溫暖貼心的朋友，善於傾聽和鼓勵。用戶剛剛分享了他們今天的心情，請給出 1-2 句溫暖的回應，然後自然地引導到下一個話題。用繁體中文回應，語調親切自然。"""
        elif self.current_question_index == 1:
            # 針對今天發生的事的回應
            system_prompt = """你是一位善解人意的朋友，用戶剛分享了今天發生的事。請給出 1-2 句有共鳴的回應，表現出你真的在聽並且關心。用繁體中文回應，語調溫暖。"""
        elif self.current_question_index == 2:
            # 針對顏色選擇的回應
            system_prompt = """用戶剛分享了代表自己的顏色。請對他們的選擇表達認同和欣賞，可以簡單描述這個顏色給你的感覺。1-2 句即可，繁體中文，語調讚美但不誇張。"""
        elif self.current_question_index == 3:
            # 針對比喻的回應
            system_prompt = """用戶用了一個很有創意的比喻來形容自己。請表達對這個比喻的欣賞，可以說說這個比喻給你的感覺。1-2 句，繁體中文，語調欣賞且溫暖。"""
        else:
            # 其他情況的通用回應
            system_prompt = """你是一位溫暖的朋友，請對用戶的分享給出 1-2 句溫暖的回應。用繁體中文，語調親切。"""

        # 記錄用戶回應
        self.user_responses.append(user_input)

        # 生成 AI 的跟進回應
        ai_response = self.get_ai_response(system_prompt, user_input)

        # 獲取下一個問題
        next_question, is_finished = self.get_next_question()

        # 組合完整回應
        if is_finished:
            full_response = ai_response + "\n\n" + next_question
            return full_response, True
        else:
            full_response = ai_response + "\n\n" + next_question
            return full_response, False

    def generate_image_prompt(self):
        """將用戶的所有回應轉換為圖像生成的 prompt"""
        if len(self.user_responses) < 3:
            return "a beautiful portrait, warm lighting, artistic style"

        # 使用 AI 來分析用戶回應並生成圖像 prompt
        system_prompt = """你是一位專業的視覺描述專家。請根據用戶在日記對話中的回應，創造一個詳細的英文圖像描述 prompt，用於 AI 圖像生成。

        要求：
        1. 結合用戶的心情、經歷、顏色偏好、自我比喻等元素
        2. 生成一個適合人像或意境畫的 prompt
        3. 包含藝術風格描述（如 portrait, artistic, beautiful 等）
        4. 保持正面積極的描述
        5. 只輸出英文 prompt，不要其他解釋

        用戶回應摘要：
        """ + "\n".join([f"問題{i+1}: {resp}" for i, resp in enumerate(self.user_responses)])

        prompt = self.get_ai_response(system_prompt, "請生成圖像描述")

        # 添加一些技術性的增強詞
        enhanced_prompt = prompt + ", masterpiece, best quality, detailed, artistic lighting, beautiful composition"

        return enhanced_prompt

# 初始化 Agent（如果 AI 客戶端可用）
if 'ai_client' in globals():
    diary_agent = DiaryAgent(ai_client)
    print("🤖 AI 日記 Agent 初始化完成！")
else:
    print("❌ AI 客戶端未就緒，請先設定 API Key")

In [ ]:
# ========== Cell 6: 圖像生成函數 ==========
def generate_diary_image(prompt, negative_prompt="blurry, bad anatomy, low quality, worst quality",
                        height=512, width=512, steps=25):
    """根據日記內容生成個人畫像"""
    try:
        # 清理記憶體
        gc.collect()
        if device == "cuda":
            torch.cuda.empty_cache()

        # 設定隨機種子
        seed = random.randint(0, 2**32 - 1)
        generator = torch.Generator(device).manual_seed(seed)

        # 生成圖像
        with torch.no_grad():
            result = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                height=height,
                width=width,
                num_inference_steps=steps,
                guidance_scale=7.5,
                generator=generator
            )

        return result.images[0], f"✅ 生成成功！使用 seed: {seed}"

    except Exception as e:
        return None, f"❌ 生成失敗: {str(e)}"

In [ ]:
# ========== Cell 7: 創建 Gradio 界面 ==========
def chat_with_agent(user_input, chat_history):
    """處理與 Agent 的對話"""
    if not user_input or not user_input.strip():
        return chat_history, ""

    # 檢查 diary_agent 是否存在
    if 'diary_agent' not in globals():
        chat_history.append([user_input, "❌ AI Agent 未初始化，請先設定 API Key"])
        return chat_history, ""

    try:
        # 處理用戶輸入
        ai_response, is_finished = diary_agent.process_user_response(user_input)

        # 更新對話歷史
        chat_history.append([user_input, ai_response])

    except Exception as e:
        error_msg = f"❌ 對話處理失敗: {str(e)}"
        chat_history.append([user_input, error_msg])

    return chat_history, ""

def generate_final_image():
    """生成最終的個人畫像"""
    # 檢查必要組件
    if 'diary_agent' not in globals():
        return None, "❌ AI Agent 未初始化"

    if 'pipe' not in globals():
        return None, "❌ Stable Diffusion 模型未載入"

    if len(diary_agent.user_responses) < 3:
        return None, "請先完成至少 3 個問題的對話 😊"

    try:
        # 生成圖像 prompt
        image_prompt = diary_agent.generate_image_prompt()

        # 生成圖像
        image, status = generate_diary_image(image_prompt)

        return image, f"🎨 根據你的分享生成的畫像\n\n使用的描述：{image_prompt}\n\n{status}"

    except Exception as e:
        return None, f"❌ 圖像生成失敗: {str(e)}"

def reset_conversation():
    """重置對話"""
    global diary_agent

    # 檢查 ai_client 是否存在
    if 'ai_client' not in globals():
        return [["", "❌ 請先設定 API Key"]], None, "請先設定 AI API"

    try:
        diary_agent = DiaryAgent(ai_client)

        # 返回初始問題
        first_question, _ = diary_agent.get_next_question()
        initial_chat = [["", first_question]]

        return initial_chat, None, "開始新的日記對話 ✨"

    except Exception as e:
        return [["", f"❌ 初始化失敗: {str(e)}"]], None, "初始化失敗"

# 自定義 CSS
custom_css = """
.gradio-container {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    font-family: 'Arial', sans-serif;
}
.chat-container {
    background: rgba(255,255,255,0.1);
    border-radius: 15px;
    padding: 20px;
    backdrop-filter: blur(10px);
}
.gr-button {
    background: linear-gradient(45deg, #FF6B6B, #4ECDC4);
    border: none;
    color: white;
    font-weight: bold;
    border-radius: 10px;
}
"""

# 創建界面
with gr.Blocks(css=custom_css, title="AI 日記繪師") as demo:

    gr.Markdown("""
    # 🎨 AI 日記繪師
    ## 📖 讓 AI 陪你寫日記，再為你畫出「我眼中的你」

    **✨ 體驗流程：**
    1. 🗣️ 與 AI 進行溫暖的日記對話 (5個問題)
    2. 🎯 AI 分析你的回應，理解今天的你
    3. 🎨 自動生成專屬的個人畫像
    4. 💕 獲得獨一無二的「我眼中的你」
    """)

    # 檢查模型狀態
    model_status = ""
    if 'pipe' in globals() and pipe is not None:
        model_status = f"✅ **Stable Diffusion 已就緒** (設備: {device.upper()})"
    else:
        model_status = "⚠️ **Stable Diffusion 模型未載入，請先執行模型載入程式碼**"

    ai_status = ""
    if 'ai_client' in globals() and ai_client is not None:
        ai_status = " | ✅ **AI Agent 已就緒**"
    else:
        ai_status = " | ❌ **AI Agent 未就緒，請設定 API Key**"

    gr.Markdown(model_status + ai_status)

    with gr.Row():
        # 左側：對話區
        with gr.Column(scale=1):
            gr.Markdown("### 💭 日記對話區")

            # 安全的初始化對話
            try:
                if 'diary_agent' in globals() and diary_agent is not None:
                    first_question, _ = diary_agent.get_next_question()
                    initial_chat = [["", first_question]]
                else:
                    initial_chat = [["", "請先設定 API Key 並初始化 AI Agent 🤖"]]
            except Exception as e:
                initial_chat = [["", f"初始化錯誤: {str(e)} 請重新執行相關程式碼"]]

            chatbot = gr.Chatbot(
                value=initial_chat,
                label="與 AI 日記助手對話",
                height=400,
                elem_classes=["chat-container"]
            )

            user_input = gr.Textbox(
                label="💬 你的回應",
                placeholder="在這裡分享你的想法...",
                lines=2
            )

            with gr.Row():
                send_btn = gr.Button("📤 發送", variant="primary")
                reset_btn = gr.Button("🔄 重新開始", variant="secondary")

        # 右側：圖像生成區
        with gr.Column(scale=1):
            gr.Markdown("### 🖼️ 我眼中的你")

            generated_image = gr.Image(
                label="AI 為你創作的畫像",
                height=400
            )

            generate_btn = gr.Button(
                "🎨 生成我的畫像",
                variant="primary",
                size="lg"
            )

            image_status = gr.Textbox(
                label="生成狀態",
                value="完成對話後，點擊上方按鈕生成專屬畫像 ✨",
                interactive=False,
                lines=4
            )

    # ========== 事件綁定 ==========

    # 發送訊息
    send_btn.click(
        fn=chat_with_agent,
        inputs=[user_input, chatbot],
        outputs=[chatbot, user_input]
    )

    # 按 Enter 發送
    user_input.submit(
        fn=chat_with_agent,
        inputs=[user_input, chatbot],
        outputs=[chatbot, user_input]
    )

    # 生成圖像
    generate_btn.click(
        fn=generate_final_image,
        outputs=[generated_image, image_status]
    )

    # 重置對話
    reset_btn.click(
        fn=reset_conversation,
        outputs=[chatbot, generated_image, image_status]
    )

    # 使用說明
    gr.Markdown("""
    ### 📋 使用指南

    **🎯 完整體驗步驟：**
    1. **開始對話**：AI 會問你 5 個溫暖的問題
    2. **真誠分享**：分享你今天的心情、經歷、想法
    3. **獲得畫像**：完成對話後點擊「生成我的畫像」
    4. **欣賞作品**：AI 會根據你的分享創作獨特畫像

    **💡 分享技巧：**
    - 真實表達你的感受，不用完美
    - 可以分享具體的細節和想法
    - 顏色和比喻可以很有創意
    - 越真誠分享，畫像越貼近你的內心

    **🔄 想重新體驗？**
    點擊「重新開始」就能開啟新的日記對話！
    """)

print("🎨 AI 日記繪師界面創建完成！")


In [ ]:
# ========== Cell 8: 啟動應用 ==========
# 執行這個 cell 來啟動完整的日記繪師系統
if __name__ == "__main__":
    print("🚀 正在啟動 AI 日記繪師...")
    demo.launch(
        share=True,
        debug=True,
        height=900,
        show_error=True
    )